# Atividade 3

Faça tudo via dbt. 

Envia como resposta:
- A query SQL
- Comando utilizado via terminal no `dbt`
- Um print do resultado no próprio notebook, para isso é necessário criar uma pasta no repositório onde serão armazenadas as imagens, e para inserir no notebook é necessário criar uma célula `markdown` e inserir o seguinte comando: 

```markdown
![Nome da Foto](path/para/imagem)
```

1. Crie uma tabela `orders_by_year` a partir de `orders` que agrupa a quantidade de pedidos por ano, considerando a coluna `order_approved_at`

> Query SQL:

```SQL
{{ config(materialized='table') }}

   SELECT
      extract(year from order_approved_at) as ano_do_pedido,
      count (order_id) as total
   FROM 
      `dbt_shop.orders` 
    WHERE
      order_approved_at IS NOT NULL
   GROUP BY 
      ano_do_pedido
```

> Comando do terminal

` dbt run --models shop.order_by_year `

> Print do Resultado

![Resposta do exercicio 1](imagens_atividade_3/exercicio_1.png)

2. Crie uma `view` que agrega os totais de pedidos por estado. Considere as tabelas _customers_ para obter os estados (coluna _customer_state_) e _orders_ para contabilizar a quantidade de pedidos.

> Query SQL:

```SQL
SELECT
    customers.customer_state as estado, 
    count(orders.order_id) as total
FROM 
    `dbt_shop.orders` as orders
INNER JOIN
    `dbt_shop.customers` as customers on (orders.customer_id = customers.customer_id)
GROUP BY customers.customer_state
```

> Comando do terminal

` dbt run --models shop.totais_pedidos_por_estado `

> Print do Resultado

![Resposta do exercicio 2](imagens_atividade_3/exercicio_2.png)

3. De acordo com o resultado do exercício 2 retorne apenas os estados que tem mais de 5000 pedidos, esse número deve ser passado via variável `number_of_orders` do `dbt`.

> Query SQL:

```SQL
SELECT
    *
FROM 
    `dataset_iago.totais_pedidos_por_estado`
WHERE
    total > CAST('{{ var("number_of_orders") }}' as INT64)
```

> Comando do terminal

` dbt run --models shop.pedidos_por_estado_com_variavel --vars "{'number_of_orders': '5000'}" `

> Print do Resultado

![Resposta do exercicio 3](imagens_atividade_3/exercicio_3.png)

4. Crie uma `view` chamada `customers_by_region` que utiliza os dados da tabela `customers`. A visão deve realizar uma agregação por região do Brasil (Norte, Nordeste, Sudeste, Centro Oeste e Sul) e contar o número de clientes em cada uma delas.

> Query SQL:

```SQL
SELECT 
    case
        when customer_state like 'AM' then 'Norte'
        when customer_state like 'RR' then 'Norte'
        when customer_state like 'AP' then 'Norte'
        when customer_state like 'PA' then 'Norte'
        when customer_state like 'TO' then 'Norte'
        when customer_state like 'RO' then 'Norte'
        when customer_state like 'AC' then 'Norte'
        when customer_state like 'RN' then 'Nordeste'
        when customer_state like 'CE' then 'Nordeste'
        when customer_state like 'MA' then 'Nordeste'
        when customer_state like 'PI' then 'Nordeste'
        when customer_state like 'PE' then 'Nordeste'
        when customer_state like 'PB' then 'Nordeste'
        when customer_state like 'SE' then 'Nordeste'
        when customer_state like 'AL' then 'Nordeste'
        when customer_state like 'BA' then 'Nordeste'
        when customer_state like 'MT' then 'Centro-Oeste'
        when customer_state like 'MS' then 'Centro-Oeste'
        when customer_state like 'GO' then 'Centro-Oeste'
        when customer_state like 'DF' then 'Centro-Oeste'
        when customer_state like 'SP' then 'Sudeste'
        when customer_state like 'RJ' then 'Sudeste'
        when customer_state like 'MG' then 'Sudeste'
        when customer_state like 'ES' then 'Sudeste'
        when customer_state like 'PR' then 'Sul'
        when customer_state like 'SC' then 'Sul'
        when customer_state like 'RS' then 'Sul'

    end as regiao,
    count (customer_id) as numero_de_clientes
FROM `dbt_shop.customers`
GROUP BY regiao
```

> Comando do terminal

` dbt run --models shop.customers_by_region `

> Print do Resultado

![Resposta do exercicio 4](imagens_atividade_3/exercicio_4.png)

5. Crie uma tabela `items_costs` que contenha o preço total de um item (soma do valor do produto com o frete).

> Query SQL:

```SQL
{{ config(materialized='table') }}

   SELECT
      order_id,
      order_item_id, 
      product_id,
      ROUND((price + freight_value), 2) as item_cost
   FROM 
      `dbt_shop.items`
```

> Comando do terminal

` dbt run --models shop.items_costs `

> Print do Resultado

![Resposta do exercicio 5](imagens_atividade_3/exercicio_5.png)

6. Crie uma `view` com o nome `orders_shipped_costs`, você deve calcular o preço total do pedido somado com o frete, apenas para pedidos enviado (shipped).

> Query SQL:

```SQL
SELECT 
    orders.order_id, 
    orders.order_status, 
    items.price, 
    items.freight_value, 
    ROUND((items.price + items.freight_value), 2) as full_price 
FROM 
    `dbt_shop.orders` as orders
INNER JOIN 
    `dbt_shop.items` as items on (items.order_id = orders.order_id)
WHERE 
    order_status like "shipped"
```

> Comando do terminal

` dbt run --models shop.orders_shipped_costs `

> Print do Resultado

![Resposta do exercicio 6](imagens_atividade_3/exercicio_6.png)